# RIC and RBDS 3D running study: an example of collaborative and reproducible research
> Reginaldo K Fukuchi$^{1,2}$, Reed Ferber$^{2}$ 
> $^{1}$Biomedical Engineering, Federal University of ABC, Brazil
> $^{2}$Faculty of Kinesiology, University of Calgary, Canada


This notebook presents the comparison of 3D running kinematics between runners from Brazil and Canada. The data were collected using similar experimental procedures and by the same experimenter (RKF) in both centres. 

The notebook uses the raw marker trajectories from standing calibration and running trials; and outputs 3D joint angles of the hip, knee and ankle joints during the gait cycle. 

All the analyses were performed using computer programs written in Python and they are provided along with the markers data and the notebook itself. Therefore, the consistency of the results presented here can be assessed by any user interested in reproducing this reseach.

The data are from published data sets (Fukuchi et al. (2017) and Ferber et al. (in preparation).

<p style="text-align: right;">A <a href="https://jupyter.org/">Jupyter Notebook</a></p>

In [1]:
# Prepare environment
import os, sys
import scipy.io as spio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
#%matplotlib notebook

sys.path.insert(1, r'./../functions')

### Supporting functions

In [75]:
def f7(seq):
    """
    This function drop duplictes in a list while keeping the sequence order of elements
    """
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def hip_jc(RASI, LASI):
    """
    Calculate hip joint centre based on Coda method. 
    See https://c-motion.com/v3dwiki/index.php?title=Coda_Pelvis
    RHJC=(-0.19*ASIS_Distance,-0.3*ASIS_Distance,0.36*ASIS_Distance)
    """
    ASIS_distance_S = np.linalg.norm(RASI-LASI, axis=1)
    hjc = np.array([RASI[:,0]-0.19*ASIS_distance_S, RASI[:,1]-0.3*ASIS_distance_S,
                  RASI[:,2]-0.36*ASIS_distance_S]).T
    return hjc

def pelvisCS(RASIr, LASIr, VSACr):
    # Pelvic reference system
    midASIS = (RASIr+LASIr)/2
    
    v2p = np.cross(RASIr-VSACr,LASIr-VSACr)
    v3p = RASIr-midASIS
    v1p = np.cross(v2p,v3p)

    v1p = v1p/np.linalg.norm(v1p)
    v2p = v2p/np.linalg.norm(v2p)
    v3p = v3p/np.linalg.norm(v3p)

    bp = np.array([v1p,v2p,v3p])
    
    return bp

def thighCS(rhjc_R, rkjc_R, RKNLr):
    # Thigh reference system
    v2t = rhjc_R-rkjc_R
    v1t = np.cross(v2t,RKNLr-rkjc_R)
    v3t = np.cross(v1t,v2t)

    v1t = v1t/np.linalg.norm(v1t)
    v2t = v2t/np.linalg.norm(v2t)
    v3t = v3t/np.linalg.norm(v3t)

    bt = np.array([v1t,v2t,v3t])
    
    return bt

def shankCS(rkjc_R, rajc_R, RMALr):
    # Shank reference system
    v2s = rkjc_R-rajc_R
    v1s = np.cross(v2s,RMALr-rajc_R)
    v3s = np.cross(v1s,v2s)

    v1s = v1s/np.linalg.norm(v1s)
    v2s = v2s/np.linalg.norm(v2s)
    v3s = v3s/np.linalg.norm(v3s)

    bs = np.array([v1s,v2s,v3s])
    
    return bs

def footCS(heelT, heelB):
    # Foot reference system
    v1f   = np.array([[1,0,0]]) # paralell to lab x-axis
    v2f_t = heelT - heelB
    v3f = np.cross(v1f, v2f_t)
    v2f = np.cross(v3f, v1f)
   
    #v1f = v1f/np.linalg.norm(v1f)
    v2f = v2f/np.linalg.norm(v2f)
    v3f = v3f/np.linalg.norm(v3f)
    
    bf = np.array([v1f,v2f,v3f])
    
    return bf

## Import data

In [3]:
data_dir = r'../data'

In [49]:
labCS = np.array([[[1,0,0]],[[0,1,0]],[[0,0,1]]])
labCS.shape

(3, 1, 3)

### Standing calibration and running trial data

In [7]:
df_S = pd.read_csv(os.path.join(data_dir, 'RBDS_static.csv'), 
                        usecols=range(1,109)) # Static markers
df_R = pd.read_csv(os.path.join(data_dir, 'RBDS_run.csv'), 
                        usecols=range(1,79)) # Dynmamic markers

In [9]:
# List of static trial markers
mkr_S_lbl = [mkr[:-2] for mkr in df_S.columns.tolist()]
mkr_S_lbl = f7(mkr_S_lbl)
# List of running trial markers
mkr_R_lbl = [mkr[:-2] for mkr in df_R.columns.tolist()]
mkr_R_lbl = f7(mkr_R_lbl)

### Calculate hip joint center
It was based on the method used in 3Dgait system from Running Injury Clinic, ie, 25% of the distance between hips.

In [20]:
R_hip = df_S[['R_hip_X','R_hip_Y','R_hip_Z']].values
L_hip = df_S[['L_hip_X','L_hip_Y','L_hip_Z']].values
rhjc_S = R_hip - (R_hip-L_hip)/4 # Right HJC
lhjc_S = L_hip + (L_hip-R_hip)/4 # Left HJC

In [35]:
# Shank reference system
rkjc_S = (df_S[['R_lat_knee_X','R_lat_knee_Y','R_lat_knee_Z']].values+
        df_S[['R_med_knee_X','R_med_knee_Y','R_med_knee_Z']].values)/2
rajc_S = (df_S[['R_lat_ankle_X','R_lat_ankle_Y','R_lat_ankle_Z']].values+
        df_S[['R_med_ankle_X','R_med_ankle_Y','R_med_ankle_Z']].values)/2

bs = shankCS(rkjc_S, rajc_S, df_S[['R_lat_ankle_X','R_lat_ankle_Y',
                                   'R_lat_ankle_Z']].values)
print('Versors of the shank:')
print(bs)

Versors of the shank:
[[[ 0.97670129 -0.03645553  0.21148426]]

 [[ 0.05076583  0.99674458 -0.06263446]]

 [[-0.20851241  0.07191133  0.97537241]]]


In [52]:
# Thigh reference system
v2t = rhjc_S-rkjc_S
v1t = np.cross(v2t,df_S[['R_lat_knee_X','R_lat_knee_Y',
                         'R_lat_knee_Z']].values-rkjc_S)
bt = thighCS(rhjc_S, rkjc_S, df_S[['R_lat_knee_X','R_lat_knee_Y',
                         'R_lat_knee_Z']].values)
print('Versors of the thigh:')
print(bt)

Versors of the thigh:
[[[ 0.99534241 -0.0944786  -0.01916483]]

 [[ 0.09313988  0.99374299 -0.06164276]]

 [[ 0.02486884  0.05957065  0.99791426]]]


In [107]:
# Pelvis reference system
# Aligned to the lab
bp = labCS
print('Versors of the pelvis:')
print(bp)

Versors of the pelvis:
[[[1 0 0]]

 [[0 1 0]]

 [[0 0 1]]]


In [78]:
# Foot reference system
bf = footCS(df_S[['R_foot_1_X', 'R_foot_1_Y', 'R_foot_1_Z']].values, 
            df_S[['R_foot_2_X', 'R_foot_2_Y', 'R_foot_2_Z']].values)
print('Versors of the foot:')
print(bf)

Versors of the foot:
[[[ 1.          0.          0.        ]]

 [[ 0.          0.99995349 -0.00964463]]

 [[-0.          0.00964463  0.99995349]]]


## Reconstruct anatomical markers using SVD algorithm

In [80]:
import sys
sys.path.insert(1, r'../functions')
from svdt import svdt

### Segments

In [81]:
# Pelvic segment

In [127]:
# Pelvis reference system
bp_S_O = df_S[['pelvis_4_X','pelvis_4_Y','pelvis_4_Z']].values # pelvic origin
bp_S = np.vstack((bp_S_O,bp_S_O+bp[:,0])) # pelvis basis static
# Pelvic basis running
bp_R = df_R[['pelvis_1_X','pelvis_1_Y','pelvis_1_Z',
             'pelvis_2_X','pelvis_2_Y','pelvis_2_Z',
             'pelvis_3_X','pelvis_3_Y','pelvis_3_Z',
             'pelvis_4_X','pelvis_4_Y','pelvis_4_Z']].values
# Reconstructed markers based on technical clusters
Rp, Lp, RMSEp = svdt(bp_S, bp_R) 

In [117]:
# Thigh segment
At = df_S[['R_thigh_1_X', 'R_thigh_1_Y', 'R_thigh_1_Z', 
           'R_thigh_2_X', 'R_thigh_2_Y', 'R_thigh_2_Z', 
           'R_thigh_3_X', 'R_thigh_3_Y', 'R_thigh_3_Z', 
           'R_thigh_4_X', 'R_thigh_4_Y', 'R_thigh_4_Z']].values
Bt = df_R[['R_thigh_1_X', 'R_thigh_1_Y', 'R_thigh_1_Z', 
           'R_thigh_2_X', 'R_thigh_2_Y', 'R_thigh_2_Z', 
           'R_thigh_3_X', 'R_thigh_3_Y', 'R_thigh_3_Z', 
           'R_thigh_4_X', 'R_thigh_4_Y', 'R_thigh_4_Z']].values
Rt, Lt, RMSEt = svdt(At, Bt)

In [119]:
# Shank segment
As = df_S[['R_shank_1_X', 'R_shank_1_Y', 'R_shank_1_Z', 
           'R_shank_2_X', 'R_shank_2_Y', 'R_shank_2_Z', 
           'R_shank_3_X', 'R_shank_3_Y', 'R_shank_3_Z', 
           'R_shank_4_X', 'R_shank_4_Y', 'R_shank_4_Z']].values
Bs = df_R[['R_shank_1_X', 'R_shank_1_Y', 'R_shank_1_Z', 
           'R_shank_2_X', 'R_shank_2_Y', 'R_shank_2_Z', 
           'R_shank_3_X', 'R_shank_3_Y', 'R_shank_3_Z', 
           'R_shank_4_X', 'R_shank_4_Y', 'R_shank_4_Z']].values
Rs, Ls, RMSEs = svdt(As, Bs)

In [120]:
print(df_R.columns.tolist())

['pelvis_1_X', 'pelvis_1_Y', 'pelvis_1_Z', 'pelvis_2_X', 'pelvis_2_Y', 'pelvis_2_Z', 'pelvis_3_X', 'pelvis_3_Y', 'pelvis_3_Z', 'pelvis_4_X', 'pelvis_4_Y', 'pelvis_4_Z', 'L_thigh_1_X', 'L_thigh_1_Y', 'L_thigh_1_Z', 'L_thigh_2_X', 'L_thigh_2_Y', 'L_thigh_2_Z', 'L_thigh_3_X', 'L_thigh_3_Y', 'L_thigh_3_Z', 'L_thigh_4_X', 'L_thigh_4_Y', 'L_thigh_4_Z', 'R_thigh_1_X', 'R_thigh_1_Y', 'R_thigh_1_Z', 'R_thigh_2_X', 'R_thigh_2_Y', 'R_thigh_2_Z', 'R_thigh_3_X', 'R_thigh_3_Y', 'R_thigh_3_Z', 'R_thigh_4_X', 'R_thigh_4_Y', 'R_thigh_4_Z', 'L_shank_1_X', 'L_shank_1_Y', 'L_shank_1_Z', 'L_shank_2_X', 'L_shank_2_Y', 'L_shank_2_Z', 'L_shank_3_X', 'L_shank_3_Y', 'L_shank_3_Z', 'L_shank_4_X', 'L_shank_4_Y', 'L_shank_4_Z', 'R_shank_1_X', 'R_shank_1_Y', 'R_shank_1_Z', 'R_shank_2_X', 'R_shank_2_Y', 'R_shank_2_Z', 'R_shank_3_X', 'R_shank_3_Y', 'R_shank_3_Z', 'R_shank_4_X', 'R_shank_4_Y', 'R_shank_4_Z', 'L_foot_1_X', 'L_foot_1_Y', 'L_foot_1_Z', 'L_foot_2_X', 'L_foot_2_Y', 'L_foot_2_Z', 'L_foot_3_X', 'L_foot_3_Y',

In [ ]:
# Pelvis reference system
bp_S_O = df_S[['pelvis_4_X','pelvis_4_Y','pelvis_4_Z']].values # pelvic origin
bp_S = np.vstack((bp_O,bp_S_O+bp[:,0])) # pelvis basis static
# Pelvic basis running
bp_R = df_R[['pelvis_1_X','pelvis_1_Y','pelvis_1_Z',
             'pelvis_2_X','pelvis_2_Y','pelvis_2_Z',
             'pelvis_3_X','pelvis_3_Y','pelvis_3_Z',
             'pelvis_4_X','pelvis_4_Y','pelvis_4_Z']].values
# Reconstructed markers based on technical clusters
Rp, Lp, RMSEp = svdt(bp_S, bp_R) 

In [130]:
bf[:,0]

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.99995349, -0.00964463],
       [-0.        ,  0.00964463,  0.99995349]])

In [136]:
bp_R.shape

(4500, 12)

# PENDING 
ADD COLUMS WITH ORIGING DYNAMIC

In [134]:
# Foot origin static markers
bf_S_O = (df_S[['R_foot_1_X', 'R_foot_1_Y', 'R_foot_1_Z']].values+ \
df_S[['R_foot_2_X', 'R_foot_2_Y', 'R_foot_2_Z']].values+ \
df_S[['R_foot_3_X', 'R_foot_3_Y', 'R_foot_3_Z']].values) / 3
bf_S = np.vstack((bf_S_O,bf_S_O+bf[:,0])) # foot basis static

# Foot basis running
bf_R = df_R[['R_foot_1_X', 'R_foot_1_Y', 'R_foot_1_Z', 'R_foot_2_X', 'R_foot_2_Y', 'R_foot_2_Z', 'R_foot_3_X', 'R_foot_3_Y', 'R_foot_3_Z']].values

In [135]:
bf_R.shape

(4500, 9)

In [122]:
df_S[['R_foot_1_X', 'R_foot_1_Y', 'R_foot_1_Z']]

,R_foot_1_X,R_foot_1_Y,R_foot_1_Z
0,638.931886,90.743773,1359.633209


In [ ]:
# Foot segment
Af = df_s[['RHED_X','RHED_Y','RHED_Z','RHEP_X','RHEP_Y','RHEP_Z',
             'RHEL_X','RHEL_Y','RHEL_Z']].values.mean(axis=0)
Bf = df_r[['RHED_X','RHED_Y','RHED_Z','RHEP_X','RHEP_Y','RHEP_Z',
             'RHEL_X','RHEL_Y','RHEL_Z']].values
Rf, Lf, RMSEf = svdt(Af, Bf)

In [ ]:
# knee markers
RKNLs = df_s[['RKNL_X','RKNL_Y','RKNL_Z']].values.mean(axis=0)
RKNMs = df_s[['RKNM_X','RKNM_Y','RKNM_Z']].values.mean(axis=0)
RKNLr = np.empty(shape=(Rt.shape[0],3))
RKNMr = np.empty(shape=(Rt.shape[0],3))
# ankle markers
RMALs = df_s[['RMAL_X','RMAL_Y','RMAL_Z']].values.mean(axis=0)
RMAMs = df_s[['RMAM_X','RMAM_Y','RMAM_Z']].values.mean(axis=0)
RMALr = np.empty(shape=(Rs.shape[0],3))
RMAMr = np.empty(shape=(Rs.shape[0],3))
# foot markers
RMH1s = df_s[['RMH1_X','RMH1_Y','RMH1_Z']].values.mean(axis=0)
RMH5s = df_s[['RMH5_X','RMH5_Y','RMH5_Z']].values.mean(axis=0)
RMH1r = np.empty(shape=(Rs.shape[0],3))
RMH5r = np.empty(shape=(Rs.shape[0],3))
# Reconstructed markers based on technical clusters
RKNLr = np.dot(Rt,RKNLs)  + Lt
RKNMr = np.dot(Rt,RKNMs)  + Lt
RMALr = np.dot(Rs,RMALs)  + Ls
RMAMr = np.dot(Rs,RMAMs)  + Ls
RMH1r = np.dot(Rf,RMH1s)  + Ls
RMH5r = np.dot(Rf,RMH5s)  + Ls
RTOEr = (RMH1r+RMH5r)/2

### Hip, knee and ankle joint centres

In [ ]:
ASIS_distanceR = np.linalg.norm(RASIr-LASIr, axis=1)
rhjc_R = np.array([RASIr[:,0]-0.19*ASIS_distanceR, RASIr[:,1]-0.3*ASIS_distanceR, RASIr[:,2]-0.36*ASIS_distanceR]).T

In [ ]:
rkjc_R = (RKNLr+RKNMr)/2
rajc_R = (RMALr+RMAMr)/2

### Visualize markers

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

#### RBDS

In [ ]:
# Average marker position
mkr_S = df_RIC_S.values.flatten()
# Marker labels list
mkr_S_lbl = RIC_mkr_S_lbl

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8, 10))
ax = fig.add_subplot(111, projection='3d',  facecolor='white')
#ax.view_init(15, 20)


ax.scatter(mkr_S[2:-1:3], mkr_S[0:-3:3], mkr_S[1:-2:3],  
           c='r', s=30, depthshade=False)
for m, mkr_lbl in enumerate(mkr_S_lbl):
    ax.text(mkr_S[3*m+2], mkr_S[3*m], mkr_S[3*m+1], mkr_lbl)

ax.set_xlabel('\n' + 'Z [m]', linespacing=2)
ax.set_ylabel('\n' + 'X [m]', linespacing=2)
ax.set_zlabel('\n' + 'Y [m]', linespacing=2)
#ax.invert_yaxis()
# square plot
ax.set_aspect('equal', adjustable='box')
ax.set_xlim3d([np.nanmin(mkr_S[2::3])-.4, np.nanmax(mkr_S[2::3])+.4])
ax.set_ylim3d([np.nanmin(mkr_S[0::3])-.4, np.nanmax(mkr_S[0::3])+.4])
ax.set_zlim3d([np.nanmin(mkr_S[1::3]), np.nanmax(mkr_S[1::3])])
plt.show()

In [ ]:
#fig.savefig(os.path.join(data_dir, 'RBDS_3d.png'))

#### RBDS
the following Lab Coordinate system
*       X - points to the driection of walking
*       Y - points vertically upwards
*       Z - points to the subject's right

#### RIC
the following Lab Coordinate system
*       X - points to the subject's right
*       Y - points vertically upwards
*       Z - points opposite of the walking direction
NOTE the Segment coordinate systems
*       X - Anterior .....................................[AB / AD duction]
*       Y - Vertically upwards ............................[Axial rotation]
*       Z - points to the subject's right side ...[Hinge flexion extension]

In [ ]:
# Average marker position. Keep the same number of mkrs than RBDS
data_RIC_static = data_RIC_static[df_S_RBDS.columns.tolist()]

In [ ]:
data_RIC_static

In [ ]:
mkr_S = data_RIC_static.values.flatten()

In [ ]:
# Marker labels list
mkr_S_lbl = [data_RIC_static.columns.tolist()[i][:-2] for i in range(mkr_S.shape[0])]
mkr_S_lbl_ = f7(mkr_S_lbl)

In [ ]:
# Static
mkrs_RIC_S = np.empty(mkr_S.shape[0])
rot = np.array([[0,0,1],[0,1,0],[-1,0,0]]) # rotate markers 90 deg
for m in range(len(mkr_S_lbl_)):
    mkrs_RIC_S[3*m:3*m+3] = rot @ mkr_S[3*m:3*m+3]

In [ ]:
mkr_S = mkrs_RIC_S

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8, 10))
ax = fig.add_subplot(111, projection='3d',  facecolor='white')
#ax.view_init(15, 20)


ax.scatter(mkr_S[0:-3:3], mkr_S[1:-2:3], mkr_S[2:-1:3], c='r', s=30, depthshade=False)
for m, mkr_lbl in enumerate(mkr_S_lbl_):
    ax.text(mkr_S[3*m], mkr_S[3*m+1], mkr_S[3*m+2], mkr_lbl)

ax.set_xlim3d([np.nanmin(mkr_S[0::3])-.4, np.nanmax(mkr_S[0::3])+.4])
ax.set_ylim3d([np.nanmin(mkr_S[1::3])-.4, np.nanmax(mkr_S[1::3])+.4])
ax.set_zlim3d([np.nanmin(mkr_S[2::3]), np.nanmax(mkr_S[2::3])])
ax.set_xlabel('\n' + 'X [m]', linespacing=2)
ax.set_ylabel('\n' + 'Y [m]', linespacing=2)
ax.set_zlabel('\n' + 'Z [m]', linespacing=2)
#ax.invert_yaxis()
# square plot
ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
data_RIC_run = pd.DataFrame(data_RIC['running'])
data_RIC_run.head()

In [ ]:
data_RIC_static.values.shape[1]/3

In [ ]:
mkr_lbl = mkr_lbl_RIC_s+mkr_lbl_RIC_j

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = plt.axes(projection='3d')

mkr_lbl = mkr_lbl_RIC_s+mkr_lbl_RIC_j
# Data for three-dimensional scattered points
for i in range(len(mkr_lbl)):
    ydata = data_RIC_static.values[0,3*i+1]
    xdata = data_RIC_static.values[0,3*i]
    zdata = data_RIC_static.values[0,3*i+2]
    ax.scatter(xdata, zdata, ydata, c='b');
    ax.text(xdata,zdata,ydata,  '%s' % (mkr_lbl[i]), size=5, zorder=1,  
    color='k')
ax.set_aspect('equal')

In [ ]:
from ordered_set import OrderedSet
# RIC
ls_RIC = df_s_RIC.columns.tolist()
mkr_S_labels_RIC = list(OrderedSet([x[:-2].replace('.','_') for x in ls_RIC]))
# RBDS
ls = df_s_RBDS.columns.tolist()
mkr_S_labels_RBDS = list(OrderedSet([x[:-2].replace('.','_') for x in ls]))

##### Running trial

In [ ]:
data_RIC_run = pd.DataFrame(data_RIC['running'])
data_RIC_run.head()

In [ ]:
data_RIC = loadmat(os.path.join(data_dir, '20140515T133244_r.mat'))

In [ ]:
data_RIC['r_angles'].keys()

In [ ]:
df_s.head()

# Running data

In [ ]:
df_r = pd.read_csv(fname_out_R, delimiter=',', index_col='Time')

In [ ]:
df_r.head()

## Gait events (TD and TO)
These gait events were detected in another NB (RIC_RBDS_gait_events_detection.ipynb) using the method of Zeni.

In [ ]:
LAB = np.array([[1,0,0],[0,1,0],[0,0,1]])

In [ ]:
def td_detect(LAB, RHEDr, RTOEr, bp):
  """
  Detect gait events using the method described by Zeni et al. ()
  """
  import sys
  sys.path.insert(1, os.path.join(drive_root,'functions'))
  from svdt import svdt
  from detecta import detect_peaks

  # Global to local transformation to obtain rotation and translation matrix
  Rp, Lp, RMSEp = svdt(LAB, bp)

  # Express the RTOE and RHEEL markers in the pelvis segment
  RHEDr_pelvis = np.empty(shape=RHEDr.shape)
  RTOEr_pelvis = np.empty(shape=RTOEr.shape)
  for ix in range(RHEDr.shape[0]):
    RHEDr_pelvis[ix,:] = np.dot(Rp[ix,:,:],RHEDr[ix,:]) + Lp[ix,:]
    RTOEr_pelvis[ix,:] = np.dot(Rp[ix,:,:],RTOEr[ix,:]) + Lp[ix,:]

  # Find touch-down time stamps
  iTD = detect_peaks(RHEDr_pelvis[:,0], mph=None, mpd=70, valley=False, show=False)

  return iTD

In [ ]:
  bp = np.empty(shape=(RTOEr.shape[0],9))
  for ix in range(RHEDr.shape[0]):
    # Pelvis basis
    pp = pelvisCS(RASIr[i,:], LASIr[i,:], VSACr[i,:])
    bp[ix,:] = pp.reshape((9,))

In [ ]:
iTD = td_detect(LAB, RHEDr, RTOEr, bp)

## Calculate knee joint angle according to Grood & Suntay (1983)

In [ ]:
def hipang(p_hip_rf, d_hip_rf):
    """
    Calculate 3D hip joint angle in degrees
    """
    bp = p_hip_rf
    bt = d_hip_rf
    angh = np.empty(shape=(3,))
    yh = bt[1,:];
    zh = bp[2,:];
    fh = np.cross(yh,zh); #floating axis
    fh = fh/np.linalg.norm(fh)
    bh =np.array([fh, yh, zh]);
    zt_t = bt[2,:];
    
    # knee angle
    angh[0] = (np.pi/2 - np.arccos(np.sum(bp[1]*fh)))*180/np.pi # Flexion/Extension
    angh[1] = (-np.pi/2 - np.arccos(np.sum(yh*bp[2])))*180/np.pi + 180 #Abduction/aduction
    angh[2] = (np.pi/2 - np.arccos(np.sum(zt_t*fh)))*180/np.pi #Internal/external rotation
    
    return angh

In [ ]:
def kneeang(proxrf, distalrf):
    """
    Calculate 3D knee joint angle in degrees
    """
    angk = np.empty(shape=(3,))
    proxrf = bt
    distalrf = bs
    yk = bs[1,:];
    zk = bt[2,:];
    fk = np.cross(yk,zk); #floating axis
    fk = fk/np.linalg.norm(fk)
    bk =np.array([fk, yk, zk]);
    zs_t = bs[2,:];
    
    # knee angle
    angk[0] = -(np.pi/2 - np.arccos(np.sum(bt[1]*fk)))*180/np.pi # Flexion/Extension
    angk[1] = (-np.pi/2 - np.arccos(np.sum(yk*bt[2])))*180/np.pi + 180 #Abduction/aduction
    angk[2] = (np.pi/2 - np.arccos(np.sum(zs_t*fk)))*180/np.pi #Internal/external rotation
    
    return angk

In [ ]:
def ankleang(p_ank_rf, d_ank_rf):
    """
    Calculate 3D ankle joint angle in degrees
    """
    bs = p_ank_rf
    bf = d_ank_rf
    anga = np.empty(shape=(3,))
    ya = bf[1,:];
    za = bs[2,:];
    fa = np.cross(ya,za); #floating axis
    fa = fa/np.linalg.norm(fa)
    ba =np.array([fa, ya, za]);
    
    # knee angle
    anga[0] = (np.pi/2 - np.arccos(np.sum(bs[1]*fa)))*180/np.pi # Flexion/Extension
    anga[1] = (-np.pi/2 - np.arccos(np.sum(ya*bs[2])))*180/np.pi + 180 #Abduction/aduction
    anga[2] = (np.pi/2 - np.arccos(np.sum(bf[2]*fa)))*180/np.pi #Internal/external rotation
    
    return anga

### Joint angles during running

In [ ]:
angh_r = np.empty(shape=(rkjc_R.shape[0],3))
angk_r = np.empty(shape=(rkjc_R.shape[0],3))
anga_r = np.empty(shape=(rkjc_R.shape[0],3))
for i in range(rkjc_R.shape[0]):
    bp = pelvisCS(RASIr[i,:],LASIr[i,:],VSACr[i,:])
    bt = thighCS(rhjc_R[i,:],rkjc_R[i,:],RKNLr[i,:])
    bs = shankCS(rkjc_R[i,:],rajc_R[i,:],RMALr[i,:])
    bf = footCS(RHEDr[i,:], RMH1r[i,:], RMH5r[i,:], RTOEr[i,:])
    # Hip angle
    angh_r[i,:] = hipang(bp, bt)
    # Knee angle
    angk_r[i,:] = kneeang(bt, bs)
    # Ankle angle
    anga_r[i,:] = ankleang(bs, bf)

In [ ]:
time = df_r.index.values

## Ensemble average curve using tnorma

In [ ]:
# When running for the first time in a given computer
#fc_path = r'/content/drive/MyDrive/Github/UFABC_UofC_datasets/functions'
#!pip install --target=$fc_path detecta
#!pip install --target=$fc_path tnorma

In [ ]:
from tnorma import tnorma

In [ ]:
ang_hip = np.empty(shape=(101,iTD.shape[0]-1,3))
ang_knee= np.empty(shape=(101,iTD.shape[0]-1,3))
for i in range(iTD.shape[0]-1):
    ang_hip[:,i,:], tn, indie = tnorma(angh_r[iTD[i]:iTD[i+1],:], k=1, 
                                       smooth=0, mask=None, show=False)
    ang_knee[:,i,:], tn, indie= tnorma(angk_r[iTD[i]:iTD[i+1],:], k=1, 
                                       smooth=0, mask=None, show=False)

# RIC and RBDS data

ang_hipM_RIC = np.mean(ang_hip, axis=1)
ang_kneeM_RIC= np.mean(ang_knee, axis=1)

In [ ]:
ang_hipM_RBDS = np.mean(ang_hip, axis=1)
ang_kneeM_RBDS= np.mean(ang_knee, axis=1)

In [ ]:
fig, axs = plt.subplots(2, figsize=(10,6))
fig.suptitle('Join angles')
axs[0].plot(tn,ang_hipM_RBDS[:,0],'b')
axs[0].set_title('RBDS')
axs[0].set_ylabel('Hip Flexion')
axs[1].plot(tn,ang_kneeM_RBDS[:,0],'b')
axs[1].set_title('RBDS')
axs[1].set_ylabel('Knee Flexion')
plt.show()

# References
* Fukuchi RK, Fukuchi CA, Duarte M. 2017. A public dataset of running biomechanics and the effects of running speed on lower extremity kinematics and kinetics. PeerJ 5:e3298 https://doi.org/10.7717/peerj.3298
* Ferber et al. (in preparation). WeTrac 3D gait data set.

# Commit changes and push them to GitHub repository

In [ ]:
!git status

In [ ]:
!git add .

In [ ]:
!git config --global user.email "reginaldo.fukuchi@gmail.com"

In [ ]:
!git config --global user.name "regifukuchi"

In [ ]:
!git commit -m "Modified RIC_RBDS_calculate_angles.ipynb"

In [ ]:
!git push